# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0
#l1_lambda_regularization = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Tensorflow Models

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

#regularizers.l2(l2_lambda_regularization)
def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y

In [4]:
def print_model_weights(model):

    for layer in model.layers:
        weights = layer.get_weights() # list of numpy arrays
        
        for weight in weights:
        
            print(weight.shape)
            print(weight)

# Create the Data Handler

In [5]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [6]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [7]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001,beta_1=0.5)
    
    #optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = test_model(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model


def tf_compiled_model(input_shape, output_shape):
    
    tf.reset_default_graph()
    X, y = create_placeholders(input_shape, output_shape)
    
    y_pred = tf_model(X)
    cost = tf.losses.mean_squared_error(y, y_pred)
    reg_cost = tf.losses.get_regularization_loss()
    total_cost = cost + reg_cost
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001,beta1=0.5).minimize(total_cost)
    
    return {'X_placeholder':X, 'y_placeholder':y, 'y_pred':y_pred, 'cost':cost, 'total_cost':total_cost, 'optimizer':optimizer}

In [8]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)
#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

# Load Data

In [9]:
tModel.data_handler.sequence_length = 24
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [10]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(18331, 336)
(18331, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.59060403 -0.0455243  -0.27982732 ... -0.45454545  0.33333333
   0.33501825]
 [-0.36912752  0.0629156  -0.18014129 ... -0.27272727  0.25581395
   0.50126369]
 [-0.23489933 -0.13299233 -0.13854003 ... -0.09090909  0.11627907
   0.46222971]
 [-0.23489933 -0.39897698 -0.2299843  ... -0.27272727  0.31782946
   0.55293457]
 [-0.22147651 -0.39590793 -0.05926217 ... -0.81818182  0.34883721
   0.09491716]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Testing data (X, y)
[[-0.10738255 -0.49616368 -0.26844584 ... -0.27272727  0.03875969
   0.29458017]
 [-0.12751678 -0.18158568  0.02197802 ... -0.63636364  0.03875969
   0.0322943 ]
 [ 0.06711409 -0.05473146  0.13814757 ...  0.09090909  0.2248062
  

In [11]:
# #Load sequenced data (do not unroll sequence into a single feature vector)

# tModel.data_handler.data_scaler = min_max_scaler
# tModel.data_scaler = None

# tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
# #tModel.data_handler.print_data()
# tModel.print_data()

In [12]:
#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = None

# Train the model and test some Tunable Model functionalities

In [23]:
nFeatures = len(selected_features)

#lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = len(selected_features)*tModel.data_handler.sequence_length
print(shape)

#modelRULSN = RULmodel_SN_5(shape)
#modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

#model = get_compiled_model(shape, model_type='ann')
#tModel.change_model('ModelRUL_SN_1', model, 'keras')

model = tf_compiled_model(shape, 1)
tModel.change_model('ModelRUL_SN_1', model, 'tensorflow')

#tModel.print_data()

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel.change_model('ModelRUL_RNN_1', model, 'keras')

#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

#print("Printing model weights")
#print_model_weights(tModel.model)

tModel.epochs = 200

sess = tf.Session()

tModel.train_model(tf_session=sess, verbose=1)

336
[[-0.59060403 -0.0455243  -0.27982732 ... -0.45454545  0.33333333
   0.33501825]
 [-0.36912752  0.0629156  -0.18014129 ... -0.27272727  0.25581395
   0.50126369]
 [-0.23489933 -0.13299233 -0.13854003 ... -0.09090909  0.11627907
   0.46222971]
 ...
 [ 0.10067114  0.58516624  0.33398744 ...  0.63636364 -0.53488372
  -0.89019938]
 [ 0.42281879  0.55498721  0.53649922 ...  0.09090909 -0.76744186
  -0.52316765]
 [ 0.4295302   0.13452685  0.43877551 ...  0.27272727 -0.64341085
  -0.55630441]]
[[129.]
 [129.]
 [129.]
 ...
 [  2.]
 [  1.]
 [  0.]]
Epoch: 0001 cost_reg= 7764.341227214 cost= 7709.717502170
Epoch: 0002 cost_reg= 4152.542816162 cost= 4092.195173475
Epoch: 0003 cost_reg= 1404.514182197 cost= 1336.794887967
Epoch: 0004 cost_reg= 731.211364746 cost= 659.210831536
Epoch: 0005 cost_reg= 520.875823975 cost= 443.142221239
Epoch: 0006 cost_reg= 463.420709398 cost= 383.181577894
Epoch: 0007 cost_reg= 435.970579359 cost= 354.225260417
Epoch: 0008 cost_reg= 417.990464952 cost= 335.116543

In [24]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, tf_session=sess)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

tensorflow test
scores
Engine 1, Predicted RUL 106.85459899902344, Rounded RUL 106.0, Real RUL 112.0
Engine 2, Predicted RUL 131.2633819580078, Rounded RUL 131.0, Real RUL 98.0
Engine 3, Predicted RUL 46.94147491455078, Rounded RUL 46.0, Real RUL 69.0
Engine 4, Predicted RUL 79.00276184082031, Rounded RUL 79.0, Real RUL 82.0
Engine 5, Predicted RUL 87.82756805419922, Rounded RUL 87.0, Real RUL 91.0
Engine 6, Predicted RUL 112.91165161132812, Rounded RUL 112.0, Real RUL 93.0
Engine 7, Predicted RUL 107.88333129882812, Rounded RUL 107.0, Real RUL 91.0
Engine 8, Predicted RUL 93.47246551513672, Rounded RUL 93.0, Real RUL 95.0
Engine 9, Predicted RUL 121.0886459350586, Rounded RUL 121.0, Real RUL 111.0
Engine 10, Predicted RUL 93.68683624267578, Rounded RUL 93.0, Real RUL 96.0
Engine 11, Predicted RUL 79.62642669677734, Rounded RUL 79.0, Real RUL 97.0
Engine 12, Predicted RUL 95.20885467529297, Rounded RUL 95.0, Real RUL 124.0
Engine 13, Predicted RUL 90.19422149658203, Rounded RUL 90.0, R